# Formatting CARB Statewide Emissions Data

## 0. Import Packages

In [1]:
import pandas as pd

## 1. Test Work on PM 2.5 Emissions

In [2]:
# load data
pm25 = pd.read_csv('../data/CARB Emissions Data/statewide/CARB_cepam_pm2_5.csv')

In [3]:
pm25.head()

,AREA,SRC_TYPE,CATEGORY,SUBCATEGORY,POLLUTANT,SEASON,CONTROL_TYPE,2000,2001,2002,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,v1.03_RF3084
0,STATEWIDE,STATIONARY,FUEL COMBUSTION,ELECTRIC UTILITIES,PM2_5,ANNUAL AVERAGE,GROWN AND CONTROLLED,7.990296,9.212034,7.435656,...,6.028129,5.407026,5.307804,4.535730,4.463215,4.236524,4.061526,4.554937,4.490335,v1.03_RF3084
1,STATEWIDE,STATIONARY,FUEL COMBUSTION,COGENERATION,PM2_5,ANNUAL AVERAGE,GROWN AND CONTROLLED,3.713187,4.270487,3.339253,...,2.341881,2.089655,2.098039,1.544696,1.571302,1.533594,1.540158,1.546744,1.553368,v1.03_RF3084
2,STATEWIDE,STATIONARY,FUEL COMBUSTION,OIL AND GAS PRODUCTION (COMBUSTION),PM2_5,ANNUAL AVERAGE,GROWN AND CONTROLLED,1.971739,1.472658,1.179439,...,2.095733,2.661281,2.656146,2.398715,2.215645,2.155201,2.096769,2.040115,1.985155,v1.03_RF3084
3,STATEWIDE,STATIONARY,FUEL COMBUSTION,PETROLEUM REFINING (COMBUSTION),PM2_5,ANNUAL AVERAGE,GROWN AND CONTROLLED,4.167303,3.592789,3.267510,...,3.876679,4.632335,4.436270,4.000298,4.144733,4.167910,4.191088,4.212930,4.236224,v1.03_RF3084
4,STATEWIDE,STATIONARY,FUEL COMBUSTION,MANUFACTURING AND INDUSTRIAL,PM2_5,ANNUAL AVERAGE,GROWN AND CONTROLLED,6.960978,7.400318,5.658637,...,4.901713,4.623426,4.499194,4.446501,4.138890,4.201592,4.202737,4.179744,4.192661,v1.03_RF3084


In [4]:
# lowercase columns; drop unnecessary columns
pm25.columns = [col.lower() for col in pm25.columns]
pm25.drop(columns=['area', 'pollutant', 'season', 'control_type', 'v1.03_rf3084'], inplace=True)

In [5]:
# melt years
pm25 = pd.melt(pm25, id_vars=['src_type', 'category', 'subcategory'], var_name='year', value_name='pm25')

In [6]:
# cast year as integer
pm25.year = [int(year) for year in pm25.year]

In [7]:
pm25

,src_type,category,subcategory,year,pm25
0,STATIONARY,FUEL COMBUSTION,ELECTRIC UTILITIES,2000,7.990296
1,STATIONARY,FUEL COMBUSTION,COGENERATION,2000,3.713187
2,STATIONARY,FUEL COMBUSTION,OIL AND GAS PRODUCTION (COMBUSTION),2000,1.971739
3,STATIONARY,FUEL COMBUSTION,PETROLEUM REFINING (COMBUSTION),2000,4.167303
4,STATIONARY,FUEL COMBUSTION,MANUFACTURING AND INDUSTRIAL,2000,6.960978
...,...,...,...,...,...
1755,MOBILE,OTHER MOBILE SOURCES,FARM EQUIPMENT,2021,3.245501
1756,MOBILE,OTHER MOBILE SOURCES,FUEL STORAGE AND HANDLING,2021,0.000000
1757,NATURAL (NON-ANTHROPOGENIC),NATURAL SOURCES,BIOGENIC SOURCES,2021,0.000000
1758,NATURAL (NON-ANTHROPOGENIC),NATURAL SOURCES,GEOGENIC SOURCES,2021,0.000000


## 2. Format All Emissions CSVs as DataFrames Stored in Dictionary

In [8]:
em_types = ['co', 'nh3', 'nox', 'pm', 'pm10', 'pm2_5', 'rog', 'sox', 'tog']
em_df_dict = {}

In [9]:
for em in em_types:
    # load data
    em_df = pd.read_csv(f'../data/CARB Emissions Data/statewide/CARB_cepam_{em}.csv')
    
    # lowercase columns; drop unnecessary columns
    em_df.columns = [col.lower() for col in em_df.columns]
    em_df.drop(columns=['area', 'pollutant', 'season', 'control_type', 'v1.03_rf3084'], inplace=True)
    
    # melt years and cast year as integer
    em_df = pd.melt(em_df, id_vars=['src_type', 'category', 'subcategory'], var_name='year', value_name=f'{em}')
    em_df.year = [int(year) for year in em_df.year]
    
    # save df in dict
    em_df_dict[f'{em}'] = em_df

## 3. Merge All Emissions into Single DataFrame

In [10]:
# start with carbon monoxide df
all_em_df = em_df_dict['co']

In [11]:
# merge df's in one at a time, matching on all keys to ensure data integrity
for em in em_types[1:]:
    all_em_df = pd.merge(all_em_df, em_df_dict[f'{em}'], on=['src_type', 'category', 'subcategory', 'year'])

In [12]:
all_em_df

,src_type,category,subcategory,year,co,nh3,nox,pm,pm10,pm2_5,rog,sox,tog
0,STATIONARY,FUEL COMBUSTION,ELECTRIC UTILITIES,2000,87.747861,5.587486,66.249218,8.883160,8.273829,7.990296,5.250657,5.706011,33.335160
1,STATIONARY,FUEL COMBUSTION,COGENERATION,2000,47.060179,2.005656,29.043289,4.224099,3.518971,3.713187,2.968590,1.755844,16.221292
2,STATIONARY,FUEL COMBUSTION,OIL AND GAS PRODUCTION (COMBUSTION),2000,18.723581,0.038666,28.960639,1.984445,1.927553,1.971739,2.540201,7.423263,25.029807
3,STATIONARY,FUEL COMBUSTION,PETROLEUM REFINING (COMBUSTION),2000,10.252644,1.380193,47.006641,4.456122,4.229585,4.167303,1.821717,12.819814,3.961066
4,STATIONARY,FUEL COMBUSTION,MANUFACTURING AND INDUSTRIAL,2000,134.593068,2.884717,87.507772,7.047467,7.149462,6.960978,4.556243,17.840740,21.225969
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,MOBILE,OTHER MOBILE SOURCES,FARM EQUIPMENT,2021,114.384736,0.071031,60.007085,3.557842,3.546327,3.245501,12.499444,0.078671,14.566349
1756,MOBILE,OTHER MOBILE SOURCES,FUEL STORAGE AND HANDLING,2021,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.995429,0.000000,11.995430
1757,NATURAL (NON-ANTHROPOGENIC),NATURAL SOURCES,BIOGENIC SOURCES,2021,0.000000,14.533996,25.824091,0.000000,0.000000,0.000000,4770.894523,0.000000,5345.125001
1758,NATURAL (NON-ANTHROPOGENIC),NATURAL SOURCES,GEOGENIC SOURCES,2021,0.000000,61.794575,0.000000,0.000000,0.000000,0.000000,24.906147,0.000000,91.757860


## 4. Save DataFrame to CSV

In [13]:
all_em_df.to_csv('../data/CARB Emissions Data/statewide/all_em_statewide.csv', index=False)